In [ ]:
import os
import time
import numpy as np
import pandas as pd

from skimage import io

import matplotlib.pyplot as plt

from CellSeg.segmentation import Segmentation
from CellSeg.analysis import cell_analysis, edge_analysis, face_analysis
%load_ext snakeviz

In [ ]:
path = "/mnt/sda1/Muscle_paper_2024/Data/24_28_72hpf/48hpf_2/"
seg_filename = "230707-MMS-wt-rac1a-mKate-48hpf_02_abels_myotome_1.tif"
pixel_size = dict(x_size = 0.1071804,
                  y_size = 0.1071804,
                  z_size = 0.5,)

# pixel_size = dict(x_size = 0.183,
#                   y_size = 0.183,
#                   z_size = 0.5,)

In [ ]:
image = io.imread(os.path.join(path, seg_filename))


In [ ]:
seg = Segmentation(image, pixel_size, path, nb_core=2)

In [ ]:
cell_plane_df= pd.read_csv(os.path.join(seg.storage_path, "cell_plane_df.csv"), index_col="Unnamed: 0")

In [ ]:
cell_plane_df

In [ ]:
# import sparse
# from tqdm.auto import tqdm
# from CellSeg import image as csimage

# cell_plane_df = pd.read_csv(os.path.join(seg.storage_path, "cell_plane_df.csv"), index_col="Unnamed: 0")
# cell_plane_df["nb_neighbor"] = -1

# # for ti in tqdm(range(len(seg.unique_id_cells)), desc="Cell", leave=True):
# for ti in [150]:
# # for c_id in seg.unique_id_cells:
#     c_id = int(seg.unique_id_cells[ti])
    
#     sp_mat = sparse.load_npz(os.path.join(seg.storage_path, "npz/" + str(c_id) + ".npz"))
#     img_cell_dil = sp_mat.todense()
#     img_cell = csimage.get_label(img_cell_dil, 1).astype("uint8")
#     img_cell_dil[img_cell_dil == 2] = 1
#     img_multi = np.multiply(img_cell_dil, seg.label_image)
    
#     nb_neighbors_plane = []
#     for z in range(img_multi.shape[0]):
#         id_unique = pd.unique(img_multi[z].flatten())
#         id_unique = np.delete(id_unique, np.where(id_unique == 0))
#         if len(id_unique)==0:
#             nb_neighbors_plane.append(np.nan)
#         else:
#             nb_neighbors_plane.append(len(id_unique))
        
#     for i, val in cell_plane_df[cell_plane_df["id_im"]==c_id].iterrows():
#         cell_plane_df.loc[i, "nb_neighbor"] = nb_neighbors_plane[int(val["z_center"])]

In [ ]:
import sparse
import joblib
from tqdm.auto import tqdm
from CellSeg import image as csimage
from CellSeg import utils as csutils

def unique_id_plane(cell_plane_df, c_id, seg):
    
    sp_mat = sparse.load_npz(os.path.join(seg.storage_path, "npz/" + str(c_id) + ".npz"))
    img_cell_dil = sp_mat.todense()
    img_cell = csimage.get_label(img_cell_dil, 1).astype("uint8")
    img_cell_dil[img_cell_dil == 2] = 1
    img_multi = np.multiply(img_cell_dil, seg.label_image)
    
    nb_neighbors_plane = []
    for z in range(img_multi.shape[0]):
        id_unique = pd.unique(img_multi[z].flatten())
        id_unique = np.delete(id_unique, np.where(id_unique == 0))
        nb_neighbors_plane.append(len(id_unique))
        
    for i, val in cell_plane_df[cell_plane_df["id_im"]==c_id].iterrows():
        cell_plane_df.loc[i, "nb_neighbor"] = nb_neighbors_plane[int(val["z_center"])]
        


cell_plane_df = pd.read_csv(os.path.join(seg.storage_path, "cell_plane_df.csv"), index_col="Unnamed: 0")
cell_plane_df["nb_neighbor"] = -1

delayed_call = [joblib.delayed(unique_id_plane)(cell_plane_df, int(c_id), seg) for c_id in
                        seg.unique_id_cells]
with csutils.tqdm_joblib(desc="more analysis", total=len(seg.unique_id_cells)) as progress_bar:
    res = joblib.Parallel(n_jobs=seg.nb_core, prefer="threads")(delayed_call)


In [ ]:
cell_plane_df

In [ ]:
cell_plane_df.to_csv(os.path.join(seg.storage_path, "cell_plane_df.csv"))